## Reading the JSON

source_data.json has clean normalized data used as the source of the matching.

We need to load it and then we want to make it a pandas DataFrame, as it will make much easier all future data transaction. This is a fixed cost we must pay once for running all the functions.

In [1]:
import json
from pprint import pprint

def extract_source_data(source_file):
    ''' Form a list of dictionaries from a file with a json doc per line''' 
    source_data = []
    with open(source_file) as f:
        for line in f:
            source_data.append(json.loads(line))
    return source_data

source_data = extract_source_data("source_data.json")
assert(len(source_data) == 11231)
pprint(source_data[:2])

[{'doctor': {'first_name': 'Dean',
             'last_name': 'Israel',
             'npi': '85103080143784778415'},
  'practices': [{'city': 'Port Demetris',
                 'lat': '-79.8757664338564',
                 'lon': '84.31253504872467',
                 'state': 'LA',
                 'street': '271 Annabelle Fort',
                 'street_2': 'Apt. 404',
                 'zip': '53549'}]},
 {'doctor': {'first_name': 'Quinton',
             'last_name': 'Mollie',
             'npi': '36233383542350521233'},
  'practices': [{'city': 'Nealville',
                 'lat': '81.37417480720865',
                 'lon': '-95.33450729432164',
                 'state': 'OR',
                 'street': '8496 Kennedi Inlet',
                 'street_2': 'Suite 815',
                 'zip': '52665-6811'},
                {'city': 'Rashadborough',
                 'lat': '69.84837521604314',
                 'lon': '87.36942972635728',
                 'state': 'UT',
                 'st

### Unwinding the json

We want to unwind the practices in order to:

- Look for number of doctors from match_file.csv in which first name, last name and full adress match with the source_data.
- Look for number of practices from match_file.csv in which the full adress match with the source_data.

In [2]:
import pandas as pd
from pandas.io.json import json_normalize

def transform_source_data(source_data):
    '''Form a DataFrame from a list of dictionaries. 
    Rename columns to follow name conventions as the csv'''
    fields_to_unwind = ["practices"]
    not_unwinded_fields_path = [["doctor", "first_name"], 
                           ["doctor", "last_name"], 
                           ["doctor", "npi"]]
    not_unwinded_naming_map = {'doctor.first_name':'first_name', 
                               'doctor.last_name':'last_name',
                               'doctor.npi':'npi'}
    return json_normalize(source_data, 
                         fields_to_unwind, 
                         not_unwinded_fields_path).rename(
                             columns = not_unwinded_naming_map)

source_unwinded_df = transform_source_data(source_data)
assert(source_unwinded_df.shape == (22443, 10))
source_unwinded_df

city                  lat                  lon state  \
0            Port Demetris    -79.8757664338564    84.31253504872467    LA   
1                Nealville    81.37417480720865   -95.33450729432164    OR   
2            Rashadborough    69.84837521604314    87.36942972635728    UT   
3          South Daronland    84.90377842497296   177.28706015725533    AK   
4          West Lonnieberg    52.12502086274685   109.12414094328233    GA   
5        Port Angieborough    89.41473074638557   -38.22151510102702    KY   
6              Nyasiaburgh   0.7514069044332956    93.56993517086102    NH   
7             Grantborough    78.53231427000821   12.229188372184922    MN   
8              East Ozella   25.541057391873352  -32.342152333557465    PA   
9                New Fredy   -4.541598251928605   -41.46795232079714    IL   
10             D'Amoreview   34.926103897363646    89.45264216496582    DE   
11             Lake Sheila     67.8952178905721   -71.54982356539455    AZ   
12             Keelingstad    -84.7669879597025    134.0653096213187    AL   
13             South Vesta   -80.29587184932748  -156.78993611303167    IA   
14             North Daija  -5.6687886642665575   -78.53360129963642    DE   
15        West Calistaside   -58.29709418783973   -90.33104012263043    ND   
16       South Breanahaven    35.61319092049271   -40.85256575227882    NY   
17         Turcotteborough  -26.725359700401064   147.06704213574403    ME   
18           West Penelope  -26.051512799700845   -142.5655398712161    OK   
19          Streichchester   14.435512034467493  -17.361548051772843    AZ   
20       Macejkovicborough   30.162134795190568   -90.50020614602697    IA   
21             Fernchester    83.27744784461902   -7.588193564781278    ID   
22           North Charles    8.386214668564563  -112.25443174905242    NM   
23              Pansymouth  -0.4096685064945831   25.263779569626422    WI   
24        Anastasiachester    72.21374078359321    -98.2078551359897    NM   
25      West Stefaniehaven   58.249231574415404    -162.223393234025    SD   
26           Kaylahchester    67.03323194744917    49.89908125056172    VA   
27             Lake Ransom  -63.897691603348164   -24.52191742362797    NV   
28           New Laurianne   -63.82884564326444   116.43600140040883    WI   
29             Hudsonburgh   -69.29809132686029  -45.748012666858955    TX   
...                    ...                  ...                  ...   ...   
22413  South Cullenchester   12.631159070844816    75.45447028291224    SC   
22414       West Ewaldfort  -25.603308779132135    -38.0461504637735    LA   
22415             Mohrbury    62.64373596892514    85.08571494725044    ME   
22416    East Madalynmouth    89.81853196623294   -87.60072329542766    AL   
22417     Port Rebecahaven   -33.26699639713477    43.86540422775258    OK   
22418     East Norrismouth    29.46565683323155  -175.20002511445128    WA   
22419       Lake Javonfort   23.843695138013558  -40.560638844743806    KS   
22420           Schoenside  -15.177036814470398    7.975215023389609    NE   
22421           New Ardith   -35.39690712940211   -70.11505860676358    NM   
22422        Sandrineville    76.74807915691656   -62.38967353104991    NH   
22423          East Jailyn   -50.46934133519033   169.64362150154523    KS   
22424              Eddtown     89.5005906229525  -169.49917729635942    MN   
22425    Port Mariettaside    17.39832400669364   145.95265422300474    CT   
22426              New Sam   1.4181278917870515    -9.43650395687854    NJ   
22427      East Arianeside    51.22778248610666  -139.79933501048328    AL   
22428          Tabithafurt   39.222583742168496   36.687829945461345    VA   
22429             Wolfland   -44.16782173630302   -83.51579584549863    IL   
22430          Schultzberg   -76.13525963165453   147.58867451411476    MD   
22431        Schusterville   -47.33489160195421  -111.93457928181452    AK   
22432           New Odessa  -62.056670213939

We can also find a slightly more complicated version that is more generic, allowing us specify several fields at once to unwind

In [3]:
import pandas as pd
from pandas.io.json import json_normalize

def get_formatted_not_unwinded_fields(source_series, fields_not_to_unwind):
    '''Form a list of lists with the path to the fields inside fields_to_unwind,
    and a dict mapping this path to the subfields 
    
    These are needed in order to use the json_normalize in pandas.io.json.
    
    Example: {"doctor": {"last_name": "Doe", "first_name": "John"}} will return
    -not_unwinded_fields_path = [["doctor", "first_name"], 
                                ["doctor", "last_name"]]
    -not_unwinded_naming_map = {"doctor.first_name": "first_name", 
                                "doctor.last_name": "last_name"}'''
    not_unwinded_fields_path = []
    not_unwinded_naming_map = {}
    for field in source_series:
        if field in fields_not_to_unwind:
            for subfield in source_series[field]:
                not_unwinded_fields_path.append([field, subfield])
                not_unwinded_naming_map["{}.{}".format(field, subfield)] = subfield
    return not_unwinded_fields_path, not_unwinded_naming_map

def transform_source_data(source_data, fields_to_unwind, fields_not_to_unwind):
    '''Form a DataFrame from a list of dictionaries. 
    Rename columns to follow name conventions as the csv'''
    if len(source_data) != 0:
        not_unwinded_fields_path, not_unwinded_naming_map = \
            get_formatted_not_unwinded_fields(source_data[0], fields_not_to_unwind)
    else:
        return None
    return json_normalize(source_data, 
                         fields_to_unwind, 
                         not_unwinded_fields_path).rename(
                             columns = not_unwinded_naming_map)

source_unwinded_df = transform_source_data(source_data, 
                                           fields_to_unwind=["practices"], 
                                           fields_not_to_unwind=["doctor"])
assert(source_unwinded_df.shape == (22443, 10))
source_unwinded_df

city                  lat                  lon state  \
0            Port Demetris    -79.8757664338564    84.31253504872467    LA   
1                Nealville    81.37417480720865   -95.33450729432164    OR   
2            Rashadborough    69.84837521604314    87.36942972635728    UT   
3          South Daronland    84.90377842497296   177.28706015725533    AK   
4          West Lonnieberg    52.12502086274685   109.12414094328233    GA   
5        Port Angieborough    89.41473074638557   -38.22151510102702    KY   
6              Nyasiaburgh   0.7514069044332956    93.56993517086102    NH   
7             Grantborough    78.53231427000821   12.229188372184922    MN   
8              East Ozella   25.541057391873352  -32.342152333557465    PA   
9                New Fredy   -4.541598251928605   -41.46795232079714    IL   
10             D'Amoreview   34.926103897363646    89.45264216496582    DE   
11             Lake Sheila     67.8952178905721   -71.54982356539455    AZ   
12             Keelingstad    -84.7669879597025    134.0653096213187    AL   
13             South Vesta   -80.29587184932748  -156.78993611303167    IA   
14             North Daija  -5.6687886642665575   -78.53360129963642    DE   
15        West Calistaside   -58.29709418783973   -90.33104012263043    ND   
16       South Breanahaven    35.61319092049271   -40.85256575227882    NY   
17         Turcotteborough  -26.725359700401064   147.06704213574403    ME   
18           West Penelope  -26.051512799700845   -142.5655398712161    OK   
19          Streichchester   14.435512034467493  -17.361548051772843    AZ   
20       Macejkovicborough   30.162134795190568   -90.50020614602697    IA   
21             Fernchester    83.27744784461902   -7.588193564781278    ID   
22           North Charles    8.386214668564563  -112.25443174905242    NM   
23              Pansymouth  -0.4096685064945831   25.263779569626422    WI   
24        Anastasiachester    72.21374078359321    -98.2078551359897    NM   
25      West Stefaniehaven   58.249231574415404    -162.223393234025    SD   
26           Kaylahchester    67.03323194744917    49.89908125056172    VA   
27             Lake Ransom  -63.897691603348164   -24.52191742362797    NV   
28           New Laurianne   -63.82884564326444   116.43600140040883    WI   
29             Hudsonburgh   -69.29809132686029  -45.748012666858955    TX   
...                    ...                  ...                  ...   ...   
22413  South Cullenchester   12.631159070844816    75.45447028291224    SC   
22414       West Ewaldfort  -25.603308779132135    -38.0461504637735    LA   
22415             Mohrbury    62.64373596892514    85.08571494725044    ME   
22416    East Madalynmouth    89.81853196623294   -87.60072329542766    AL   
22417     Port Rebecahaven   -33.26699639713477    43.86540422775258    OK   
22418     East Norrismouth    29.46565683323155  -175.20002511445128    WA   
22419       Lake Javonfort   23.843695138013558  -40.560638844743806    KS   
22420           Schoenside  -15.177036814470398    7.975215023389609    NE   
22421           New Ardith   -35.39690712940211   -70.11505860676358    NM   
22422        Sandrineville    76.74807915691656   -62.38967353104991    NH   
22423          East Jailyn   -50.46934133519033   169.64362150154523    KS   
22424              Eddtown     89.5005906229525  -169.49917729635942    MN   
22425    Port Mariettaside    17.39832400669364   145.95265422300474    CT   
22426              New Sam   1.4181278917870515    -9.43650395687854    NJ   
22427      East Arianeside    51.22778248610666  -139.79933501048328    AL   
22428          Tabithafurt   39.222583742168496   36.687829945461345    VA   
22429             Wolfland   -44.16782173630302   -83.51579584549863    IL   
22430          Schultzberg   -76.13525963165453   147.58867451411476    MD   
22431        Schusterville   -47.33489160195421  -111.93457928181452    AK   
22432           New Odessa  -62.056670213939

### Without unwinding

In case we want to look for number of doctors from match_file.csv in which the npi matches with the source_data, we canot have the unwinding in place for practices.

If we use the unwinded dataframe and N is the length of the list of practices for a doctor where we match the npi, we would have N matches. 

In other usecases not part of the assignment, we have to be careful when using the unwinded version to not repeat the doctors.

In [4]:
def transform_source_data_without_unwinding(source_data, fields_not_to_unwind):
    '''Form a DataFrame from a list of dictionaries. 
    Rename columns to follow name conventions as the csv'''
    if len(source_data) != 0:
        not_unwinded_naming_map = \
            get_formatted_not_unwinded_fields(source_data[0], fields_not_to_unwind)[1]
    else:
        return None
    
    return json_normalize(source_data).rename(
        columns = not_unwinded_naming_map)

source_not_unwinded_df = transform_source_data_without_unwinding(source_data, ["doctor"])
source_not_unwinded_df

first_name   last_name                   npi  \
0           Dean      Israel  85103080143784778415   
1        Quinton      Mollie  36233383542350521233   
2        Vincent       Abbie  68951826121607537145   
3        Gerardo       Piper  92442805782715742535   
4           Dean   Francesco  83029151715578341587   
5       Marshall        Cole  18233577393219566041   
6         Lawson    Lilliana  78792788275411915642   
7           Coty        Brad  50391514247237749255   
8          Billy     Gennaro  10032670447666263763   
9          Deion         Mae  36556623055822736995   
10          Zena     Deondre  02861119266722247430   
11        Dallin     Madyson  52535882133589110715   
12        Emelia      Dahlia  63048319452018546857   
13         Clyde       Mayra  87766177950596441331   
14        Dannie       Aimee  12489887534072365547   
15       Martine       Kiana  23583155472740817761   
16        Leatha      Freida  33162918725434354575   
17        Justyn       Abbie  78362387662864903554   
18        Trever   Florencio  66493161023332634050   
19         Zoila     Gillian  24419704957549334014   
20         Herta      Jeremy  35005052287178763631   
21       Rahsaan      Andrew  52531654649932861882   
22        Donato     Assunta  69045183358539837470   
23        Alison  Marguerite  23073193115632509846   
24          Nona      German  84446415756341782880   
25         Brain      Lamont  31167442866831223211   
26        Dallas     Addison  15824097234826712536   
27        Zakary       Alisa  48615383334662251587   
28          Lela       Helga  14183870164437438716   
29      Isabella      Margot  50628265194460258741   
...          ...         ...                   ...   
11201       Ford    Angelina  97289891984667859066   
11202     Virgie       Jorge  50576258968369523160   
11203     Carley      Robert  65293237204808172649   
11204        Gay     Pauline  21475631140740262661   
11205    Braxton     Deshaun  23082447513674783797   
11206    Laurine       Susan  53391868664236558025   
11207  Margarita       Abdul  31826755176318909333   
11208     Raquel     Willard  44943827535217353108   
11209    Norbert       Kraig  38694922351628224543   
11210      Bryon       Bryce  45351124443073625143   
11211     Stevie      Sienna  12346292525117756558   
11212      Nolan     Jeffrey  53534201412731144562   
11213   Julianne    Geoffrey  16775256723377963285   
11214      Erika       Halie  10716464554476538689   
11215      Cesar      Bailee  54703704821828676767   
11216      Josie        Myrl  52586272052413404321   
11217     Shania     Kaleigh  51157506582703742741   
11218   Sheridan    Domenica  68125474664901872227   
11219   Yessenia     Gaetano  11653686656447570414   
11220      Kitty       Velma  37286807935421536604   
11221     August     Jeramie  84958780385529716247   
11222   Leopoldo     Dominic  18237532451153392571   
11223      Brown      Meggie  73291346948336413140   
11224    Whitney        Luis  12745874446123256447   
11225      Maiya     Ryleigh  52605316565885025487   
11226      Bulah       Skyla  67073887994871626664   
11227       Nash        Coby  79538487122854242275   
11228       Alan        Neil  31846358102754904675   
11229       Arlo       Blake  69445612947443312384   
11230      Clare      Jensen  97541207981689217839   

                                               practices  
0      [{'zip': '53549', 'lon': '84.31253504872467', ...  
1      [{'zip': '52665-6811', 'lon': '-95.33450729432...  
2      [{'zip': '59104', 'lon': '109.12414094328233',...  
3      [{'zip': '39283', 'lon': '12.229188372184922',...  
4      [{'zip': '43570', 'lon': '-41.46795232079714',...  
5      [{'zip': '85355', 'lon': '-71.54982356539455',...  
6      [{'zip': '30997-4476', 'lon': '-78.53360129963...  
7      [{'zip': '53565', 'lon': '-90.33104012263043',...  
8      [{'zip': '81096-8100', 'lon': '-142.5655398712...  
9      [{'zip': '58023', 'lon': '-17.361548051772843'...  
10     [

## Reading the csv

match_file.csv contains raw source data that needs to be parsed and normalized


In [5]:
import pandas as pd

# match_file.csv: Raw source data that needs to be parsed and normalized.
raw_data_df = pd.read_csv("match_file.csv")

raw_data_df

first_name    last_name                   npi  \
0         Ruthe      Laverne  44843147983186317848   
1      Marshall         Cole  18233577393219566041   
2        Lawson     Lilliana  78792788275411915642   
3       Martine        Kiana  23583155472740817761   
4        Leatha       Freida                   NaN   
5        Justyn        Abbie  78362387662864903554   
6     Granville       Benton  17871640342222098849   
7        Brenda        Lenna  88137148807320232511   
8       Juliana     Benedict                   NaN   
9       Marjory       Ulices  13251241236387155567   
10        Celia        Joany  53517451823105334497   
11      Kathryn        Shany  18615056044429228990   
12     Isabelle         Nils  41196810585374325420   
13      Johnnie    Johnathon                   NaN   
14     Madaline      Shirley  34629375530320352466   
15       Khalid      Crystel  82216555626836774716   
16      Heather        Tracy  75216887016624818206   
17        Audra      Imogene                   NaN   
18      Montana      Dominic  48581605458856137841   
19      Justice        Jessy  55734468672668247847   
20     Maryjane        Sasha  53273831484766113262   
21        Brice  Maximillian  58552116202772184856   
22       Sylvia         Obie                   NaN   
23      Deborah        Jolie  72248738248131276601   
24      Marjory       Harley  85666817362462284195   
25         Zita       Yasmin  57219924682871422273   
26     Victoria        Kaleb                   NaN   
27      Pascale        Ryder  75137145868784228122   
28        Nella     Sheridan  63158792827720528132   
29        Albin       Yvette  60813239314886505555   
...         ...          ...                   ...   
1235        Art   Alejandrin  78791823677627337226   
1236     Tierra        Quinn  01734158001471915148   
1237     Shania        Alice  26851680110146908993   
1238      Aisha       Dortha                   NaN   
1239  Geraldine       Junior  16812475474743167993   
1240     Holden         Leon  85231883333620805644   
1241       Tyra     Benedict  15666128412219616349   
1242       Wade          Geo                   NaN   
1243      Oscar     Mohammad  72569943054714521155   
1244   Geovanni         Coby  67022257323561577850   
1245     Trycia       Gerald  87852185438622118572   
1246     Payton       Briana  67927006875389289539   
1247    Octavia   Alessandro                   NaN   
1248      Keven          Rex  20835255763874961451   
1249   Jonathon       Carter  41477047772454684144   
1250       Eden       Bonnie  90299473567815437531   
1251   Faustino       Kaylee                   NaN   
1252     Reuben         Arne  63803857220464706613   
1253      Einar     Missouri  16678380905253577683   
1254       Reta      Daniela  28406129481424315728   
1255   Reymundo        Tiara  26886272452725577492   
1256       Mara      Hermina                   NaN   
1257   Clarissa     Felicity  58626307266823047332   
1258      Gavin         Rita  88336286361567162510   
1259   Stanford        Reece  13481236896147722710   
1260    Zackery      Gillian  11240664612428757450   
1261       Maye        Lucie                   NaN   
1262      Nolan      Jeffrey  53534201412731144562   
1263      Cesar       Bailee  54703704821828676767   
1264    Whitney         Luis  12745874446123256447   

                          street   street_2                city state  \
0             569 glenda islands  suite 163            willport    nj   
1           59944 adaline harbor   apt. 862         keelingstad    al   
2              36175 amina mount   apt. 256         north daija    de   
3                 188 walsh flat   apt. 891         yasmeenstad    nv   
4           43796 gutmann plains  suite 341            vonmouth    fl   
5                            NaN        NaN                 NaN   NaN   
6                 95496 dare rue  suite 203         octaviastad    il   
7              361 justyn meadow  suite 635        steuberhaven    la   

We can see that there are values that are NaN, so we need to be careful.

In addition, some string fields don't have the appropriate capitalization. Let's fix that. 

In [6]:
def transform_match_file(df, fields_to_title_case, fields_to_upper_case):
    '''Change strings of certain DataFrame columns to title case and upper case'''
    for field in fields_to_upper_case:
        df[field] = df[field].apply(lambda x: x.title() if isinstance(x, str) else x)
        
    for field in fields_to_title_case:
        df[field] = df[field].apply(lambda x: x.upper() if isinstance(x, str) else x)    


fields_to_title_case = ["state"]
fields_to_upper_case = ["street", "street_2", "city"]

transform_match_file(raw_data_df, fields_to_title_case, fields_to_upper_case)

raw_data_df

first_name    last_name                   npi  \
0         Ruthe      Laverne  44843147983186317848   
1      Marshall         Cole  18233577393219566041   
2        Lawson     Lilliana  78792788275411915642   
3       Martine        Kiana  23583155472740817761   
4        Leatha       Freida                   NaN   
5        Justyn        Abbie  78362387662864903554   
6     Granville       Benton  17871640342222098849   
7        Brenda        Lenna  88137148807320232511   
8       Juliana     Benedict                   NaN   
9       Marjory       Ulices  13251241236387155567   
10        Celia        Joany  53517451823105334497   
11      Kathryn        Shany  18615056044429228990   
12     Isabelle         Nils  41196810585374325420   
13      Johnnie    Johnathon                   NaN   
14     Madaline      Shirley  34629375530320352466   
15       Khalid      Crystel  82216555626836774716   
16      Heather        Tracy  75216887016624818206   
17        Audra      Imogene                   NaN   
18      Montana      Dominic  48581605458856137841   
19      Justice        Jessy  55734468672668247847   
20     Maryjane        Sasha  53273831484766113262   
21        Brice  Maximillian  58552116202772184856   
22       Sylvia         Obie                   NaN   
23      Deborah        Jolie  72248738248131276601   
24      Marjory       Harley  85666817362462284195   
25         Zita       Yasmin  57219924682871422273   
26     Victoria        Kaleb                   NaN   
27      Pascale        Ryder  75137145868784228122   
28        Nella     Sheridan  63158792827720528132   
29        Albin       Yvette  60813239314886505555   
...         ...          ...                   ...   
1235        Art   Alejandrin  78791823677627337226   
1236     Tierra        Quinn  01734158001471915148   
1237     Shania        Alice  26851680110146908993   
1238      Aisha       Dortha                   NaN   
1239  Geraldine       Junior  16812475474743167993   
1240     Holden         Leon  85231883333620805644   
1241       Tyra     Benedict  15666128412219616349   
1242       Wade          Geo                   NaN   
1243      Oscar     Mohammad  72569943054714521155   
1244   Geovanni         Coby  67022257323561577850   
1245     Trycia       Gerald  87852185438622118572   
1246     Payton       Briana  67927006875389289539   
1247    Octavia   Alessandro                   NaN   
1248      Keven          Rex  20835255763874961451   
1249   Jonathon       Carter  41477047772454684144   
1250       Eden       Bonnie  90299473567815437531   
1251   Faustino       Kaylee                   NaN   
1252     Reuben         Arne  63803857220464706613   
1253      Einar     Missouri  16678380905253577683   
1254       Reta      Daniela  28406129481424315728   
1255   Reymundo        Tiara  26886272452725577492   
1256       Mara      Hermina                   NaN   
1257   Clarissa     Felicity  58626307266823047332   
1258      Gavin         Rita  88336286361567162510   
1259   Stanford        Reece  13481236896147722710   
1260    Zackery      Gillian  11240664612428757450   
1261       Maye        Lucie                   NaN   
1262      Nolan      Jeffrey  53534201412731144562   
1263      Cesar       Bailee  54703704821828676767   
1264    Whitney         Luis  12745874446123256447   

                          street   street_2                city state  \
0             569 Glenda Islands  Suite 163            Willport    NJ   
1           59944 Adaline Harbor   Apt. 862         Keelingstad    AL   
2              36175 Amina Mount   Apt. 256         North Daija    DE   
3                 188 Walsh Flat   Apt. 891         Yasmeenstad    NV   
4           43796 Gutmann Plains  Suite 341            Vonmouth    FL   
5                            NaN        NaN                 NaN   NaN   
6                 95496 Dare Rue  Suite 203         Octaviastad    IL   
7              361 Justyn Meadow  Suite 635        Steuberhaven    LA   

# RESULTS

## Doctor Match by NPI

We do an inner join on the npis from the two dataframes, getting only the docs that have the same npis.

The merge is O(M+N), if M is the number of rows in the json and N is the number of rows in the csv. 

Therefore, the total number of documents scanned is 2 * O(N+M) (so O(N+M)) 

- I scan everything for creating the DataFrames O(N+M)
- I scan O(N+M) documents for doing the merging.

In [7]:
def number_of_doctor_matches_by_npi(left_df, right_df):
    '''Return the number of doctor matches by npi.
    This is done through the inner merge of two dataframes on the key npi.
    The number of rows is the number of matches'''
    return pd.merge(left_df, right_df, how="inner", on="npi").shape[0]
    
print("Number of matches: {}".format(
    number_of_doctor_matches_by_npi(source_not_unwinded_df, raw_data_df)))

Number of matches: 864


In [8]:
import timeit

time_to_run_100 = timeit.timeit(
    'number_of_doctor_matches_by_npi(source_not_unwinded_df, raw_data_df)', 
    setup="from __main__ import number_of_doctor_matches_by_npi, source_not_unwinded_df, raw_data_df", 
    number=100)

print("Time to run number_of_doctor_matches_by_npi once {}".format(time_to_run_100/100))

Time to run number_of_doctor_matches_by_npi once 0.0043228287599959


We need to take into account that we are also transforming the data in order to do this that fast (although we only need to it once for similar operations). This operation is much, much slower:

In [9]:
time_to_run_100 = timeit.timeit(
    'transform_source_data_without_unwinding(source_data, ["practices"])', 
    setup="from __main__ import transform_source_data_without_unwinding, source_data", 
    number=100)

print("Time to transform the data without unwinding once {}".format(time_to_run_100/100))

Time to transform the data without unwinding once 0.6187990002699917


If our only intereset was this part of the assignment, we don't need to do the processing of the json. This function is as good and it is much faster (2 orders of magnitude, 8 ms vs 560 ms in my computer to run all 100 times).

In [10]:
def number_of_matches_by_npi(source_data, raw_data_df):
    '''Return the number of doctor matches by npi.
    -First extract all the npis from the json, and 
    insert them in a test for fast lookup (O(1) on average).
    -Sum the number of elements in the raw df whose npi is in the set'''
    source_npis = {x["doctor"]["npi"] for x in source_data}
    return raw_data_df["npi"].apply(lambda x: x in source_npis).sum()

number_of_matches_by_npi(source_data, raw_data_df)

864

In [11]:
time_to_run_100 = timeit.timeit(
    'number_of_matches_by_npi(source_data, raw_data_df)', 
    setup="from __main__ import number_of_matches_by_npi, source_data, raw_data_df", 
    number=100)

print("Time to run number_of_matches_by_npi (set implementation) once {}".format(
    time_to_run_100/100))

Time to run number_of_matches_by_npi (set implementation) once 0.006905441229991993


## Doctor Match by first name, last name and full adress

We do an inner join on the first name, the last name and the full address from the two dataframes, getting only the docs that have the same.

The merge is O(M+N), if M is the number of rows in the json and N is the number of rows in the csv. 

Therefore, the total number of documents scanned is 2 * O(N+M) (so O(N+M)) 

- We scan everything for creating the DataFrames O(N+M)
- We scan O(N+M) documents for doing the merging.

In [12]:
def number_of_doctor_matches_by_name_and_full_address(left_df, right_df):
    '''Return the number of doctor matches by 
    first_name, last_name, street, street_2, city, state and zip.
    This is done through the inner merge of two dataframes on said keys.
    The number of rows is the number of matches'''
    return pd.merge(left_df, 
                    right_df, 
                    how="inner", 
                    on=["first_name", "last_name", "street", 
                        "street_2", "city", "state", "zip"]).shape[0]


print("Number of matches by name and address: {}".format(
    number_of_doctor_matches_by_name_and_full_address(source_unwinded_df, raw_data_df)))

Number of matches by name and address: 912


In [13]:
time_to_run_100 = timeit.timeit(
    'number_of_doctor_matches_by_name_and_full_address(source_unwinded_df, raw_data_df)', 
    setup="from __main__ import number_of_doctor_matches_by_name_and_full_address, \
        source_unwinded_df, raw_data_df", 
    number=100)

print("Time to run number_of_doctor_matches_by_name_and_full_address once {}".format(
    time_to_run_100/100))

Time to run number_of_doctor_matches_by_name_and_full_address once 0.02485857120998844


Similarly, we need to take into account the time it took to do the transformation. However, this time to do the transformation should be shared between this and getting the number of practice matches by full address.

In [14]:
time_to_run_100 = timeit.timeit(
    'transform_source_data(source_data, \
        fields_to_unwind=["practices"], \
        fields_not_to_unwind=["doctor"])', 
    setup="from __main__ import transform_source_data, source_data", 
    number=100)

print("Time to run transform_source_data once {}".format(time_to_run_100/100))

Time to run transform_source_data once 0.1332419852699968


## Practice Match by full address

Very similar to the previous case, but we do an inner join only on the full address from the two dataframes, getting only the docs that have the same full address. 

The number of documents scanned will be similar, O(N+M) - although a little bit less because we are doing joins in two fields less. This can also be noted in the time it takes to run one function and the other (around 5 times less for number_of_practices_by_full_address)

In [15]:
def number_of_practices_by_full_address(left_df, right_df):
    '''Return the number of practices matches by 
    street, street_2, city, state and zip.
    This is done through the inner merge of two dataframes on said keys.
    The number of rows is the number of matches'''
    return pd.merge(left_df, 
                    right_df, 
                    how="inner", 
                    on=["street", "street_2", "city", "state", "zip"]).shape[0]
    
print("Number of matches: {}".format(
    number_of_practices_by_full_address(source_unwinded_df, raw_data_df)))

Number of matches: 912


In [16]:
time_to_run_100 = timeit.timeit(
    'number_of_practices_by_full_address(source_unwinded_df, raw_data_df)', 
    setup="from __main__ import number_of_practices_by_full_address, \
                                                source_unwinded_df,\
                                                raw_data_df", 
    number=100)

print("Time to run number_of_practices_by_full_address once {}".format(
    time_to_run_100/100))

Time to run number_of_practices_by_full_address once 0.023602791230005096


## Number of documents that could not be matched

The assumption is that the problem statement talks about the number of rows from the match_file.csv that could not be matched in source_data.json by none of the criteria above.

If the doctor is matched by full name and address, the practice will match by the address (since it is a subset of the conditions).

Therefore, we only need to see the intersection between the number of documents that don't match by npi and the numbers of documents that don't match by full address, this will be the number of documents that do not produce a match.

For the number of documents that don't match by doctor's npi, we can do an outer merge on the npi between the dataframes from the source data and from the data to match, with an indicator that will indicate if the npi column was found on one of the two dataframes or on both.

The rows of the merged table that were only in the data to match (raw_data_df) are the rows with the npis that did not match from the raw_data_df (with some extra columns and some columns named in a slightly different way due to the merge). Therefore, if we take those rows, and those columns, we will have a df that is exactly the original raw_data_df, minus the matches.

In [17]:
merged_by_npi_df = pd.merge(raw_data_df, 
                            source_not_unwinded_df, 
                            indicator=True, 
                            how="outer", 
                            on="npi")
not_matched_by_npi_df = merged_by_npi_df[merged_by_npi_df['_merge'] == 'left_only']
print("Number of documents not matched by npi {}".format(not_matched_by_npi_df.shape[0]))

Number of documents not matched by npi 401


In [18]:
selected_columns = ["first_name_x", 
                    "last_name_x", 
                    "npi", 
                    "street",
                    "street_2", 
                    "city", 
                    "state", 
                    "zip"]
column_rename_map = {"first_name_x": "first_name", "last_name_x": "last_name"}
not_matched_by_npi_df = not_matched_by_npi_df[selected_columns].rename(columns = 
                                                                       column_rename_map)

not_matched_by_npi_df

first_name    last_name                   npi                   street  \
0           Ruthe      Laverne  44843147983186317848       569 Glenda Islands   
4          Leatha       Freida                   NaN     43796 Gutmann Plains   
5         Juliana     Benedict                   NaN      798 Katarina Street   
6         Johnnie    Johnathon                   NaN            541 Nora Hill   
7           Audra      Imogene                   NaN    9228 Rodriguez Knolls   
8          Sylvia         Obie                   NaN                      NaN   
9        Victoria        Kaleb                   NaN     79341 Destin Springs   
10         Erling    Ellsworth                   NaN       72949 Wyman Valley   
11          Floyd        Toney                   NaN         133 Kris Corners   
12       Destiney       Wilson                   NaN          191 Jacobs Mill   
13          Mario      Richard                   NaN                      NaN   
14          Edgar        Deven                   NaN       5645 Earlene Coves   
15         Dewitt      Madisen                   NaN           7453 Kuhn Flat   
16        Kenneth        Tracy                   NaN          323 Boyle Roads   
17           Koby        Kaley                   NaN      188 Wilfredo Lights   
18        Nichole         Veda                   NaN                      NaN   
19           Lily      Wilburn                   NaN     43274 Scottie Bypass   
20          Mario         Amya                   NaN     1733 Jacobs Mountain   
21       Louvenia        Twila                   NaN     5495 Jakubowski Fort   
22      Monserrat        Ryder                   NaN      573 Earnest Prairie   
23          Steve       Fausto                   NaN                      NaN   
24    Christopher        Rusty                   NaN       9241 Katelyn Walks   
25          Logan      Brionna                   NaN    4987 Della Throughway   
26         Damian       Jaquan                   NaN       206 Abshire Points   
27       Lauretta    Zechariah                   NaN     4696 Dakota Crossing   
28         Xander        Neoma                   NaN                      NaN   
29          Dasia         Else                   NaN         59466 Mohr Light   
30          Chyna     Michaela                   NaN        214 Keebler Inlet   
31       Mckenzie        Jayne                   NaN       37481 Rodrick Isle   
32           Kale      Taurean                   NaN          926 Towne Brook   
...           ...          ...                   ...                      ...   
1007        Lacey       Kelton  63617075957465480655      22892 Nicolas Court   
1017         Colt       Foster  45718517812902142423         32349 Ryley Mews   
1029      Jameson       Cordie  25086269276885857815       206 Beahan Passage   
1038      Edgardo         Toby  36404641733234746671  74192 Winfield Motorway   
1045      Shakira    Francisco  68565586692773954878      39481 Lemke Freeway   
1054      Forrest        Duane  71775449690542881514         568 Tatyana Loaf   
1064       Sydney      Savanna  20353976574577626422         1243 Fahey Ports   
1071   Evangeline     Jennyfer  21742142407248195218          1995 Lane Ridge   
1084      Johanna        Louie  66775369339428044565         145 Ramona Court   
1091     Precious         Greg  55784087899398614015       258 Prohaska Grove   
1098         Judy       Hayden  87494583528254313882    8762 Vonrueden Plains   
1104      Frankie        Santa  46841105124939385261          4455 Bud Bridge   
1111        Ivory       Furman  42436382461887090811    11554 Brooklyn Forges   
1125      Jaunita  Christopher  21841154495440664139     53123 Janessa Unions   
1132       Colten        Mandy  45896655716210352578          524 Ericka Wall   
1142   Alejandrin       Cleora  66149852273568174583       6121 Marvin Radial   
1152    Josephine        Rhoda  33828474963829425666    61483 Reichel Gateway   
1159        Helga         Leon  143055

Same could be said about the number of documents that don't match by practice's full address, although the outer merge is done on the practice's full address instead of just the npi.

In [19]:
merged_by_address_df = pd.merge(raw_data_df, 
                                source_unwinded_df, 
                                indicator=True, 
                                how="outer", 
                                on=["street", "street_2", "city", "state", "zip"])
not_matched_by_address_df = merged_by_address_df[merged_by_address_df['_merge'] == 'left_only']
print("Number of documents not matched by address {}".format(not_matched_by_address_df.shape[0]))

Number of documents not matched by address 353


In [20]:
selected_columns = ["first_name_x", 
                    "last_name_x", 
                    "npi_x", 
                    "street", 
                    "street_2", 
                    "city", 
                    "state", 
                    "zip"]
column_rename_map = {"first_name_x": "first_name", "last_name_x": "last_name", "npi_x": "npi"}

not_matched_by_address_df = not_matched_by_address_df[selected_columns].rename(columns = 
                                                                               column_rename_map)
not_matched_by_address_df

first_name    last_name                   npi                   street  \
0          Ruthe      Laverne  44843147983186317848       569 Glenda Islands   
5         Justyn        Abbie  78362387662864903554                      NaN   
6          Celia        Joany  53517451823105334497                      NaN   
7        Heather        Tracy  75216887016624818206                      NaN   
8         Sylvia         Obie                   NaN                      NaN   
9        Pascale        Ryder  75137145868784228122                      NaN   
10        Millie        Amani  53407188811357432743                      NaN   
11         Elvis        Lenna  14455777372842761255                      NaN   
12         Mario      Richard                   NaN                      NaN   
13        Estell       Harvey  34268632282866767982                      NaN   
14         Colin        Tyson  85641555627354036243                      NaN   
15       Janelle         Alda  70751252862153222461                      NaN   
16       Nichole         Veda                   NaN                      NaN   
17        Selmer      Tatyana  62166944226746504643                      NaN   
18      Percival      Kennith  96816844467518308169                      NaN   
19        Lavada         Lupe  14855436163755273300                      NaN   
20         Steve       Fausto                   NaN                      NaN   
21       Sabryna       George  15412750423358644086                      NaN   
22          Lulu         Glen  58903280102772465415                      NaN   
23          Maye        Edwin  22287588043488697879                      NaN   
24        Xander        Neoma                   NaN                      NaN   
25      Marshall       Shawna  88913181517750245534                      NaN   
26        Lester         Seth  22572344054705233851                      NaN   
27      Bernardo     Penelope  68284945001639086634                      NaN   
28        Marisa     Gertrude                   NaN                      NaN   
29        Myrtie        Vidal  78252385511824852862                      NaN   
30        Amelia       Jeramy  09713575331677150777                      NaN   
31       Candice        Macie  98667557275836748133                      NaN   
32      Madelynn        Celia                   NaN                      NaN   
33        Cydney     Garfield  76762187451983822785                      NaN   
...          ...          ...                   ...                      ...   
1016     Jameson       Cordie  25086269276885857815       206 Beahan Passage   
1025     Edgardo         Toby  36404641733234746671  74192 Winfield Motorway   
1033     Shakira    Francisco  68565586692773954878      39481 Lemke Freeway   
1042     Forrest        Duane  71775449690542881514         568 Tatyana Loaf   
1053      Sydney      Savanna  20353976574577626422         1243 Fahey Ports   
1061  Evangeline     Jennyfer  21742142407248195218          1995 Lane Ridge   
1069      Samara      Mitchel  35458126372533217887        682 Vincenzo Fort   
1074     Johanna        Louie  66775369339428044565         145 Ramona Court   
1081    Precious         Greg  55784087899398614015       258 Prohaska Grove   
1088        Judy       Hayden  87494583528254313882    8762 Vonrueden Plains   
1095     Frankie        Santa  46841105124939385261          4455 Bud Bridge   
1102       Ivory       Furman  42436382461887090811    11554 Brooklyn Forges   
1117     Jaunita  Christopher  21841154495440664139     53123 Janessa Unions   
1122       Kacey       German  22434910842367773948       336 Mcglynn Valley   
1125      Colten        Mandy  45896655716210352578          524 Ericka Wall   
1135  Alejandrin       Cleora  66149852273568174583       6121 Marvin Radial   
1146   Josephine        Rhoda  33828474963829425666    61483 Reichel Gateway   
1154       Helga         Leon  14305547882001736528   69794 Bennett Villages   
1168  

If we now do an inner merge on both resulting dataframes (not_matched_by_npi_df and not_matched_by_address_df) on all columns, we will have the documents that did not match in any of the cases.

In [21]:
documents_not_matched_df = pd.merge(not_matched_by_npi_df, 
                                    not_matched_by_address_df, 
                                    how="inner", 
                                    on=list(not_matched_by_npi_df.columns))
print("Total number of documents not matched {}".format(documents_not_matched_df.shape[0]))
documents_not_matched_df

Total number of documents not matched 174


first_name    last_name                   npi                   street  \
0         Ruthe      Laverne  44843147983186317848       569 Glenda Islands   
1        Sylvia         Obie                   NaN                      NaN   
2         Mario      Richard                   NaN                      NaN   
3       Nichole         Veda                   NaN                      NaN   
4         Steve       Fausto                   NaN                      NaN   
5        Xander        Neoma                   NaN                      NaN   
6        Marisa     Gertrude                   NaN                      NaN   
7      Madelynn        Celia                   NaN                      NaN   
8          Tyra        Korey                   NaN                      NaN   
9         Cielo        Dolly                   NaN                      NaN   
10       German       Bessie                   NaN                      NaN   
11       Alivia      Cecilia                   NaN                      NaN   
12         Emil      Ephraim                   NaN                      NaN   
13        Taryn    Bridgette                   NaN                      NaN   
14       Darrel       Junior                   NaN                      NaN   
15       Allene        Reina                   NaN         231 Kiehn Courts   
16       Peyton      Rachael                   NaN                      NaN   
17          Lue       Reanna                   NaN                      NaN   
18       Hilton       Robert                   NaN                      NaN   
19       Hobart      Madelyn                   NaN                      NaN   
20       Alivia        Joany                   NaN                      NaN   
21      Zachery       Zakary                   NaN                      NaN   
22      Gregory      Hermann                   NaN                      NaN   
23      Abraham        Arvel                   NaN                      NaN   
24      Rasheed        Hilma                   NaN                      NaN   
25      Brennan        Orion                   NaN                      NaN   
26         Mona   Evangeline                   NaN                      NaN   
27       Jeanie        Chris                   NaN      4035 Mitchell Vista   
28     Mathilde       Kaylee                   NaN                      NaN   
29       Norris       Mallie                   NaN                      NaN   
..          ...          ...                   ...                      ...   
144       Lacey       Kelton  63617075957465480655      22892 Nicolas Court   
145        Colt       Foster  45718517812902142423         32349 Ryley Mews   
146     Jameson       Cordie  25086269276885857815       206 Beahan Passage   
147     Edgardo         Toby  36404641733234746671  74192 Winfield Motorway   
148     Shakira    Francisco  68565586692773954878      39481 Lemke Freeway   
149     Forrest        Duane  71775449690542881514         568 Tatyana Loaf   
150      Sydney      Savanna  20353976574577626422         1243 Fahey Ports   
151  Evangeline     Jennyfer  21742142407248195218          1995 Lane Ridge   
152     Johanna        Louie  66775369339428044565         145 Ramona Court   
153    Precious         Greg  55784087899398614015       258 Prohaska Grove   
154        Judy       Hayden  87494583528254313882    8762 Vonrueden Plains   
155     Frankie        Santa  46841105124939385261          4455 Bud Bridge   
156       Ivory       Furman  42436382461887090811    11554 Brooklyn Forges   
157     Jaunita  Christopher  21841154495440664139     53123 Janessa Unions   
158      Colten        Mandy  45896655716210352578          524 Ericka Wall   
159  Alejandrin       Cleora  66149852273568174583       6121 Marvin Radial   
160   Josephine        Rhoda  33828474963829425666    61483 Reichel Gateway   
161       Helga         Leon  14305547882001736528   69794 Bennett Villages   
162      Adolph      Mariano  74331487356298770272     

If we make this logic into a function and then proceed to timeit:

In [22]:
def number_of_documents_not_matched_by_doctors_npi(raw_data_df, source_not_unwinded_df):
    '''Return the number of documents not matched by doctor's npi'''
    merged_by_npi_df = pd.merge(raw_data_df, 
                                source_not_unwinded_df, 
                                indicator=True, 
                                how="outer", 
                                on="npi")
    #choose only the elements that did not match from the raw_data_df
    not_matched_by_npi_df = merged_by_npi_df[merged_by_npi_df['_merge'] == 'left_only']
    selected_columns_npi_df = ["first_name_x", 
                               "last_name_x",  
                               "npi", 
                               "street", 
                               "street_2", 
                               "city", 
                               "state", 
                               "zip"]
    column_rename_map_npi_df = {"first_name_x": "first_name", 
                                "last_name_x": "last_name"}
    not_matched_by_npi_df = not_matched_by_npi_df[selected_columns_npi_df]
    return not_matched_by_npi_df.rename(columns = column_rename_map_npi_df)


def number_of_documents_not_matched_by_practices_npi(raw_data_df, source_unwinded_df):
    '''Return the number of documents not matched by practice's full address'''
    merged_by_address_df = pd.merge(raw_data_df, 
                                    source_unwinded_df, 
                                    indicator=True, 
                                    how="outer", 
                                    on=["street", "street_2", "city", "state", "zip"])
    not_matched_by_address_df = merged_by_address_df[merged_by_address_df['_merge'] == 'left_only']
    selected_columns_address_df = ["first_name_x", 
                                   "last_name_x", 
                                   "npi_x", 
                                   "street", 
                                   "street_2", 
                                   "city", 
                                   "state", 
                                   "zip"]
    column_rename_map_address_df = {"first_name_x": "first_name", 
                                    "last_name_x": "last_name", 
                                    "npi_x": "npi"}
    not_matched_by_address_df = not_matched_by_address_df[selected_columns_address_df]
    return not_matched_by_address_df.rename(columns = column_rename_map_address_df)

def number_of_documents_not_matched(raw_data_df, source_unwinded_df, source_not_unwinded_df):
    '''Return the number of documents not matched by
    - Doctor by npi
    - Doctor by name and full address
    - Practices by full adress'''
    
    not_matched_by_npi_df = number_of_documents_not_matched_by_doctors_npi(raw_data_df, 
                                                                           source_not_unwinded_df)
    
    not_matched_by_address_df = number_of_documents_not_matched_by_practices_npi(raw_data_df, 
                                                                                 source_unwinded_df)
    
    return pd.merge(not_matched_by_npi_df, 
                    not_matched_by_address_df, 
                    how="inner", 
                    on=list(not_matched_by_npi_df.columns)).shape[0]

print("Return total number of documents not matched {}".format(
    number_of_documents_not_matched(raw_data_df, source_unwinded_df, source_not_unwinded_df)))

Return total number of documents not matched 174


In [23]:
time_to_run_100 = timeit.timeit(
    'number_of_documents_not_matched(raw_data_df, source_unwinded_df, source_not_unwinded_df)', 
    setup="from __main__ import \
        number_of_documents_not_matched, raw_data_df, source_unwinded_df, source_not_unwinded_df", 
    number=100)

print("Time to run number_of_documents_not_matched once {}".format(
    time_to_run_100/100))

Time to run number_of_documents_not_matched once 0.10475097256001391


The total number of documents scanned is:

- O(N+M) for the first merge + O(N+M) for getting only only the unmatched documents.
- O(N+M) for the second merge + O(N+M) for getting only only the unmatched documents.
- O(N+N) for the third merge (as the tables will be smaller or equal than the one in the csv)

Therefore, O(N+M) documents scanned.

(Remember that M is the number of rows in the json and N is the number of rows in the csv)

# Brute force approach 

We can also do all of this by brute force, but it is much much slower. However, it will be left here as a sanity check. **It does not follow good coding conventions because it is not intended to be part of the final solution, just a check**.

It takes more than 200 seconds in my machine, being 3 orders of magnitude more than doing it with pandas merges.

In [24]:
def brute_force_approach():
    doctor_matches_by_npi = 0
    practice_matches_by_full_address = 0
    doctor_matches_by_name_and_address = 0
    total_matched_documents = 0

    def check_row_for_matches(df_row):
        nonlocal doctor_matches_by_npi 
        nonlocal practice_matches_by_full_address
        nonlocal doctor_matches_by_name_and_address
        nonlocal total_matched_documents
        for record in source_data:
            npi_match = False
            practice_match = False
            if record["doctor"]["npi"] == df_row["npi"]:
                doctor_matches_by_npi += 1
                npi_match = True
            for practice in record["practices"]:
                full_address = ["street", "street_2", "city", "state", "zip"]
                if all(practice[x] == df_row[x] for x in full_address):
                    practice_matches_by_full_address += 1
                    practice_match = True
                    if record["doctor"]["first_name"] == df_row["first_name"] and \
                        record["doctor"]["first_name"] == df_row["first_name"]:
                            doctor_matches_by_name_and_address += 1
                    break
            if practice_match or npi_match:
                total_matched_documents += 1
                break
    
    raw_data_df.apply(check_row_for_matches, axis = 1)
    print("Doctor matches by npi {}".format(doctor_matches_by_npi))
    print("Practice matches by full address {}".format(practice_matches_by_full_address))
    print("Doctor matches by name and address {}".format(doctor_matches_by_name_and_address))
    print("Total matched documents {}".format(total_matched_documents))
    print("Total unmatched documents {}".format(raw_data_df.shape[0]-total_matched_documents))

brute_force_approach()

Doctor matches by npi 864
Practice matches by full address 912
Doctor matches by name and address 912
Total matched documents 1091
Total unmatched documents 174


In [25]:
time_to_run_1 = timeit.timeit(
    'brute_force_approach()', 
    setup="from __main__ import brute_force_approach", 
    number=1)

print("Time to run brute_force_approach once {}".format(time_to_run_1))

Doctor matches by npi 864
Practice matches by full address 912
Doctor matches by name and address 912
Total matched documents 1091
Total unmatched documents 174
Time to run brute_force_approach once 224.9311066400005
